In [1]:
import pandas as pd
import re

In [87]:
polugodie = pd.read_excel("Полугодия.xlsx", index_col="year")
u_sin = pd.read_excel("У-син.xlsx", index_col="Орган")
season_qi = pd.read_excel("season_qi.xlsx", index_col="Орган")
qi = pd.read_excel("qi.xlsx", index_col="Стихия")
stvoly = pd.read_excel("stvoly.xlsx", index_col="year")
vetvi = pd.read_excel("vetvi.xlsx", index_col="year")
sloy = pd.read_excel("sloy.xlsx", index_col="year")
table = pd.read_csv("table.csv", index_col='Орган')
tochki_pitanie = pd.read_csv("tochki_pitanie.csv", index_col='Орган').drop('Unnamed: 0', axis=1)
channels_Ke = pd.read_csv("channels_Ke.csv", index_col=0)


In [90]:
channels_Ke

,Gb,Si,Th,St,Co,Bl,Liv,Ht,Hg,Sp,Lu,Kid
по стволам,Sp,Lu,-,Kid,Liv,Ht,Co,Bl,-,Gb,Si,St
по ветвям,Ht,Liv,Sp,Hg,Kid,Lu,Si,Gb,St,Th,Bl,Co
по у-син,Lu,Kid,Kid,Liv,"Hg, Ht",Sp,Co,Bl,Bl,Gb,"Th, Si",St
внутри дома,Liv,"Ht, Hg","Ht, Hg",Sp,Lu,Kid,Gb,"Si, Th","Si, Th",St,Co,Bl
